# [Build vector stores and retrievers](https://python.langchain.com/docs/tutorials/retrievers/)

- [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)

In [2]:
from langchain_core.documents import Document

documents = [
    # page_content: コンテンツを表す文字列
    # metadata: 任意のメタデータを含む辞書
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

# ベクトル

ベクトル検索は、非構造化データ (非構造化テキストなど) を保存および検索するための一般的な方法です。  
テキストに関連付けられた数値ベクトルを保存し、クエリが与えられた場合、それを同じ次元のベクトルとして埋め込み、ベクトル類似性メトリックを使用してストア内の関連データを識別できます。


LangChain VectorStoreオブジェクトには、テキストやDocumentオブジェクトをストアに追加したり、様々な類似度メトリクスを使ってクエリしたりするメソッドが含まれています。  
多くの場合、埋め込みモデルで初期化され、テキストデータを数値ベクトルに変換する方法を決定します。

LangChainには、様々なベクターストア技術との統合スイートが含まれています。  
ベクターストアはクラウドプロバイダによってホストされるものだったり、Postgresのようにローカルまたはサードパーティ経由で実行できる別のインフラで実行されます。  
ここでは、インメモリ実装を含むChromaを使ったLangChain VectorStoresの使い方をデモします。

## Embeddings

テキストや単語、文、さらには画像やその他のデータを、機械学習モデルが理解しやすい数値のベクトル（数値の配列）として表現したものを指します。


## リファレンス

- [OpenAIEmbeddings](https://python.langchain.com/api_reference/openai/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)


In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


vectorstore = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

In [4]:
# 文字列クエリとの類似性に基づいてドキュメント取得
vectorstore.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [5]:
# 非同期クエリ
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [6]:
# 類似度スコアを一緒に返す
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.3749317526817322),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.4830246865749359),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.49583205580711365),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.49752357602119446)]

In [7]:
# Embeddingsされたクエリとの類似度に基づいてドキュメントを返す
embedding = OpenAIEmbeddings().embed_query("cat")
vectorstore.similarity_search_by_vector(embedding)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

# Retrievers

LangChain VectorStoreオブジェクトはRunnableのサブクラスではないので、すぐにLangChain Expression Languageチェーンに組み込むことはできませんが、  
RunnablesのサブクラスであるLangChainのRetrieversでラップすることで、LCELチェーンに組み込めるようになります。

- **Runnable**
  - 特定の処理やタスクを実行可能なオブジェクトやコンポーネントを指します。
  - LangChainは、チェーンとして複数のステップを組み合わせて自然言語処理タスクを実行するフレームワークであり、各ステップが個別の処理単位として「実行可能」な状態にあることが求められます。

## リファレンス

- [RunnableLambda](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableLambda.html)
  - PythonのCallableオブジェクトをRunnableに変換する
- [RunnablePassthrough](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)
  - 入力を変更せずに、または追加のキーを付けてパススルーするためのRunnable

In [8]:
# vectorstoreをRunnableLambdaでラップして実行
from langchain_core.runnables import RunnableLambda
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

# invokeと異なり、複数のクエリを一度に処理することができる
retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [9]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough

# モデル
model = ChatOpenAI(model="gpt-4o-mini")

# ユーザーの質問をベクトルストアからの類似性検索結果を使って回答するプロンプト
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate([
    HumanMessagePromptTemplate.from_template(message)
])

# ベクトルストアを使って類似性検索を行うリトリーバー
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result
# ユーザーの質問をそのまま渡す
question = RunnablePassthrough()


# ユーザーの質問をベクトルストアからの類似性検索結果を使って回答するチェーン
rag_chain = {"context": retriever, "question": question} | prompt | model

# Runnableの入出力タイプを表示
print(f"retriever: InputType={retriever.InputType}, OutputType={retriever.OutputType}")
print(f"question: InputType={question.InputType}, OutputType={question.OutputType}")
print(f"prompt: InputType={prompt.InputType}, OutputType={prompt.OutputType}")
print(f"model: InputType={model.InputType}, OutputType={model.OutputType}")

retriever: InputType=str, OutputType=List[Document]
question: InputType=typing.Any, OutputType=typing.Any
prompt: InputType=<class 'dict'>, OutputType=typing.Union[langchain_core.prompt_values.StringPromptValue, langchain_core.prompt_values.ChatPromptValueConcrete]
model: InputType=typing.Union[str, langchain_core.prompt_values.StringPromptValue, langchain_core.prompt_values.ChatPromptValueConcrete, list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')

In [10]:
# チェーンを実行
response = rag_chain.invoke("tell me about cats")
print(response.content)

Cats are independent pets that often enjoy their own space.
